In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('Final Train.csv')
df_comp = pd.read_csv('Final Test.csv')
df

,row_id,date,num_sold,month,year,country_Finland,country_Norway,country_Sweden,store_KaggleMart,store_KaggleRama,product_Kaggle Hat,product_Kaggle Mug,product_Kaggle Sticker,day of week_Friday,day of week_Monday,day of week_Saturday,day of week_Sunday,day of week_Thursday,day of week_Tuesday,day of week_Wednesday
0,0,2015-01-01,329,1,2015,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0
1,1,2015-01-01,520,1,2015,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0
2,2,2015-01-01,146,1,2015,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0
3,3,2015-01-01,572,1,2015,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0
4,4,2015-01-01,911,1,2015,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,26293,2018-12-31,823,12,2018,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0
26294,26294,2018-12-31,250,12,2018,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0
26295,26295,2018-12-31,1004,12,2018,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0
26296,26296,2018-12-31,1441,12,2018,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0


In [3]:
X = df.drop(columns=['row_id', 'date', 'num_sold'])
y = df[['num_sold']]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

In [5]:
from sklearn.ensemble import RandomForestRegressor

In [6]:
rf = RandomForestRegressor()

In [7]:
rf.fit(X_train, y_train)

<ipython-input-7-168a6fc83696>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


RandomForestRegressor()

In [8]:
prediction = rf.predict(X_test)

In [9]:
rf.score(X_test, y_test)

0.8972657853694819

# Comp Prediction

In [11]:
comp_pred = rf.predict(df_comp.drop(columns=['date', 'row_id']))

In [12]:
comp_pred = np.around(comp_pred)

In [13]:
df_final = df_comp

In [14]:
df_final['num_sold'] = comp_pred
df_final['num_sold'] = df_final['num_sold'].astype(int)

In [15]:
df_final = df_final[['row_id', 'num_sold']]

In [16]:
df_final.to_csv('Algo Submission.csv', index=False)

.797